In [24]:
import os
import sys
import pathlib
import subprocess
import glob

# logic to accomodate Google Colab
try:
    import google.colab

    ENV_IS_CL = True

    root = pathlib.Path("/content/t-route").resolve()
    subprocess.run(
        [
            "git",
            "clone",
            "https://github.com/NOAA-OWP/t-route.git",
        ]
    )
    
    ! pip install geopandas
    ! pip install netcdf4

except:
    ENV_IS_CL = False
    root = pathlib.Path("..").resolve()

routing_v02_dir = os.path.join(root, "src", "python_routing_v02","fast_reach")
sys.path.append(routing_v02_dir)

framework_v02_dir = os.path.join(root, "src", "python_framework_v02")
sys.path.append(framework_v02_dir)

framework_v01_dir = os.path.join(root, "src", "python_framework_v01")
sys.path.append(framework_v01_dir)

# scientific packages
import time
import xarray as xr
import pandas as pd
import numpy as np
from functools import partial
from itertools import chain, islice
import matplotlib.pyplot as plt
from joblib import delayed, Parallel

# t-route functions
import nhd_network_utilities_v02 as nnu
import nhd_io
import nhd_network
import network_dl
import mc_reach

In [180]:
# specify the routing model timestep
#----------------------------------------------------------------------#
dt = 5*60  # routing simulation timestep (seconds) 
nts = 100

# specify supernetwork connections set
#----------------------------------------------------------------------#
test_folder = os.path.join(root, r"test")
geo_input_folder = os.path.join(test_folder, r"input", r"geo")
supernetwork = "CapeFear_FULL_RES"

network_data = nnu.set_supernetwork_data(
    supernetwork=supernetwork, geo_input_folder=geo_input_folder
)

# if the NHDPlus RouteLink file does not exist, download it.
if not os.path.exists(network_data["geo_file_path"]):
    filename = os.path.basename(network_data["geo_file_path"])
    network_dl.download(network_data["geo_file_path"], network_data["data_link"])

# select only the necessary columns of geospatial data, set the DataFrame index
cols = [v for c, v in network_data["columns"].items()]
data = nhd_io.read(network_data["geo_file_path"])
data = data[cols]
data = data.set_index(network_data["columns"]["key"]).astype("int")

# mask NHDNetwork to isolate test network of choice
if "mask_file_path" in network_data:
    data_mask = nhd_io.read_mask(
        network_data["mask_file_path"], layer_string=network_data["mask_layer_string"],
    )
    data = data.filter(data_mask.iloc[:, network_data["mask_key"]], axis=0)

# sort index
data = data.sort_index()

# replace downstreams
data = nhd_io.replace_downstreams(data, network_data["columns"]["downstream"], 0)

# add a dt column to the data DataFrame
data["dt"] = dt

# rename columns to specific variable names expected by mc_reach.compute_network
column_rename = {
    network_data["columns"]["dx"]: "dx",
    network_data["columns"]["tw"]: "tw",
    network_data["columns"]["twcc"]: "twcc",
    network_data["columns"]["bw"]: "bw",
    network_data["columns"]["ncc"]: "ncc",
    network_data["columns"]["s0"]: "s0",
    network_data["columns"]["cs"]: "cs",
    network_data["columns"]["n"]: "n",
}
data = data.rename(columns=column_rename)


# extract downstream connections for each node
connections = nhd_network.extract_connections(data, network_data["columns"]["downstream"])

# organize network into reaches
#----------------------------------------------------------------------#

# reverse the network - track upstream connections
rconn = nhd_network.reverse_network(connections)

# isolate independent subnetworks
subnets = nhd_network.reachable_network(rconn)

# identify the segments in each subnetwork
subreachable = nhd_network.reachable(rconn)

# break each subnetwork into reaches
subreaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    subreaches[tw] = nhd_network.dfs_decomposition(net, path_func)

# Load Lateral Inflows
#----------------------------------------------------------------------#
# path = os.path.join(root, "test","input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_CHRTOUT")
# qlat_files = glob.glob(path + "/*", recursive=True)
# ql = nhd_io.get_ql_from_wrf_hydro(qlat_files,index_col="feature_id")

def step_qlats(data, nsteps, qlat):

    q1 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q2 = np.full((len(data.index), nsteps // 10), qlat, dtype="float32")
    q3 = np.full((len(data.index), nsteps // 10), qlat, dtype="float32")
    q4 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q5 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q6 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q7 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q8 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q9 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q10 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")

    q = np.concatenate((q1, q2, q3, q4, q5, q6, q7, q8, q9, q10), axis=1)

    ql = pd.DataFrame(q, index=data.index, columns=range(nsteps))

    return ql

# create the lateral inflow data
ql = step_qlats(data, nts, 10.0)

# Load Initial Conditions
#----------------------------------------------------------------------#
# wrf_hydro_channel_restart_file = os.path.join(root, "test", "input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_RESTART",
#                     "HYDRO_RST.2017-12-31_06-00_DOMAIN1")
# wrf_hydro_channel_ID_crosswalk_file = os.path.join(root, "test", "input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","primary_domain",
#                     "DOMAIN","Route_Link.nc")
# wrf_hydro_channel_ID_crosswalk_file_field_name = "link"

# q0 = nhd_io.get_stream_restart_from_wrf_hydro(
#     wrf_hydro_channel_restart_file,
#     wrf_hydro_channel_ID_crosswalk_file,
#     wrf_hydro_channel_ID_crosswalk_file_field_name
# )
q0 = pd.DataFrame(10,index = data.index, columns = ["qu0","qd0","h0"], dtype = "float32")

# Build Time Domain
#----------------------------------------------------------------------#
# dt_routing = pd.Timedelta(str(dt) + 'seconds')
# wrf_time = ql.columns.astype("datetime64[ns]")
# dt_wrf = (wrf_time[1] - wrf_time[0])
# sim_duration = (wrf_time[-1] + dt_wrf) - wrf_time[0]
# nts = round(sim_duration / dt_routing)

# Create order-based groupings
#----------------------------------------------------------------------#
tuple_reaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    tuple_reaches[tw] = nhd_network.dfs_decomposition_depth_tuple(net, path_func)

# convert tuple_reaches (dict) to a (list)
overall_tuple_reaches = []
for keys, tuple_list in tuple_reaches.items():
    overall_tuple_reaches.extend(tuple_list)

# create a dict with key = reach order, and values are segments in each reach of the corresponding order
overall_ordered_reaches_dict = nhd_network.tuple_with_orders_into_dict(overall_tuple_reaches)
max_order = max(overall_ordered_reaches_dict.keys())

overall_ordered_reaches_list = []
ordered_reach_count = []
ordered_reach_cache_count = []
for o in range(max_order,-1,-1):
    overall_ordered_reaches_list.extend(overall_ordered_reaches_dict[o])
    ordered_reach_count.append(len(overall_ordered_reaches_dict[o]))
    ordered_reach_cache_count.append(sum(len(r) for r in overall_ordered_reaches_dict[o]))
    
rconn_ordered = {}
rconn_ordered_byreach = {}
for o in range(max(overall_ordered_reaches_dict.keys()),-1,-1):
    rconn_ordered[o] = {}
    for reach in overall_ordered_reaches_dict[o]:
        for segment in reach:
            rconn_ordered[o][segment] = rconn[segment]
            rconn_ordered_byreach[segment] = rconn[segment]
            
# change variables to type float32, as expected by mc_reach.compute_network
data = data.astype("float32")

In [191]:
t0 = time.time()    
for i in range(max(overall_ordered_reaches_dict.keys()),-1,-1):
    
        for r in overall_ordered_reaches_dict[i]:
            
            data_sub = data.loc[
                r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
            ].sort_index()
            qlat_sub = ql.loc[r].sort_index()
            q0_sub = q0.loc[r].sort_index()
            mc_reach.compute_network(
                    nts,
                    [r],
                    {key: [] for key in r}, # this needs to be the reverse connections dictionary, keys are the segments in r 
                    data_sub.index.values.astype("long"),
                    data_sub.columns.values,
                    data_sub.values,
                    qlat_sub.values,
                    q0_sub.values,
                )


tf = time.time()
print(tf - t0)    

17.401489734649658


In [192]:
t0 = time.time()    
for i in range(max(overall_ordered_reaches_dict.keys()),-1,-1):
    
    with Parallel(n_jobs=-1, backend="threading") as parallel:
        jobs = []
        for r in overall_ordered_reaches_dict[i]:
            
            data_sub = data.loc[
                r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
            ].sort_index()
            qlat_sub = ql.loc[r].sort_index()
            q0_sub = q0.loc[r].sort_index()
            jobs.append(
                delayed(mc_reach.compute_network)(
                    nts,
                    [r],
                    {key: [] for key in r}, # this needs to be the reverse connections dictionary, keys are the segments in r 
                    data_sub.index.values.astype("long"),
                    data_sub.columns.values,
                    data_sub.values,
                    qlat_sub.values,
                    q0_sub.values,
                )
            )
            
        results = parallel(jobs)

tf = time.time()
print(tf - t0)    

55.61048102378845


In [193]:
results = []
t0 = time.time()   
for twi, (tw, reach) in enumerate(subreaches.items(), 1):
    r = list(chain.from_iterable(reach))
    data_sub = data.loc[
        r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
    ].sort_index()
    qlat_sub = ql.loc[r].sort_index()
    q0_sub = q0.loc[r].sort_index()
    results.append(
        mc_reach.compute_network(
            nts,
            reach,
            subnets[tw],
            data_sub.index.values,
            data_sub.columns.values,
            data_sub.values,
            qlat_sub.values,
            q0_sub.values,
        )
    )
tf = time.time()
print(tf - t0)  

1.8971974849700928


In [3]:
def reach_timeloop(nts, reach, rconn, flowveldepth, ql, data, q0, assume_short_ts):
    
    # loop through timesteps
    timestep = 0
    while timestep < nts:

        #--------------------------------------------------#
        # extract upstream boundary conditions: qup and quc 
        #--------------------------------------------------#
        quc = 0.0
        qup = 0.0
        for i in rconn[reach[0]]:
            quc += flowveldepth.loc[i,timestep].q
            if timestep > 0:
                qup += flowveldepth.loc[i,(timestep-1)].q
            else:
                qup += q0.loc[i].qd0

        if assume_short_ts:
                quc = qup

        boundary = np.array([quc,qup], dtype = "float32")

        #--------------------------------------------------#
        # initialize parameter input array
        #--------------------------------------------------#
        parameter_inputs = np.zeros((len(reach), 10), dtype = "float32")

        #--------------------------------------------------#
        # fill parameter input array with lateral inflow data
        #--------------------------------------------------#
        parameter_inputs[:,0] = ql.loc[reach].iloc[:,(int(timestep/(nts/ql.shape[1])))]

        #--------------------------------------------------#
        # fill parameter input array with MC parameter data
        #--------------------------------------------------#
        parameter_inputs[:,1:10] = data[["dt","dx","bw","tw","twcc","n","ncc","cs","s0"]].loc[reach].to_numpy()

        #--------------------------------------------------#
        # fill buffer with initial conditions: qdp and velp 
        #--------------------------------------------------#
        previous_state = np.zeros((len(reach), 3), dtype = "float32")
        if timestep > 0:
            previous_state[:,0:3] = flowveldepth.loc[reach,(timestep-1)].to_numpy()
        else:
            previous_state[:,0] = q0.loc[reach].qd0
            previous_state[:,2] = q0.loc[reach].h0

        #--------------------------------------------------#
        # initalize output buffer
        #--------------------------------------------------#
        output_buffer = np.zeros((len(reach), 3), dtype = "float32")

        #--------------------------------------------------#
        # compute reach
        #--------------------------------------------------#
        output_buffer = mc_reach.compute_reach(boundary, previous_state, parameter_inputs, output_buffer, assume_short_ts)

        #--------------------------------------------------#
        # update flowveldepth with computation results
        #--------------------------------------------------#
        flowveldepth.loc[reach,timestep] = np.asarray(output_buffer).tolist()

        timestep += 1
    

In [45]:
from joblib import delayed, Parallel

# initialize the flowveldepth array
fdv_columns = pd.MultiIndex.from_product([range(nts), ["q", "v", "d"]])
flowveldepth = pd.DataFrame(0, index=data.index, columns=fdv_columns, dtype = "float32")

# short timestep assumption?
assume_short_ts = True

t1 = time.time()
# loop through groups
for i in range(max(overall_ordered_reaches_dict.keys()),-1,-1):
            
    for reach in overall_ordered_reaches_dict[i]:

        reach_timeloop(nts, reach, rconn, flowveldepth, ql, data, q0, assume_short_ts)

t2 = time.time()

print(t2 - t1)


38.51084804534912
